In [5]:
%pip install tqdm
%pip install datasets
%pip install pandas
import tqdm as notebook_tqdm
from datasets import load_dataset
import pandas as pd

In [6]:
#Load dataset
mbr_data = load_dataset('recmeapp/mobilerec', data_dir='interactions')
#mbr_meta = load_dataset('recmeapp/mobilerec', data_dir='app_meta')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
mbr_data

DatasetDict({
    train: Dataset({
        features: ['app_package', 'review', 'rating', 'votes', 'date', 'uid', 'formated_date', 'unix_timestamp', 'app_category'],
        num_rows: 19297019
    })
})

In [8]:
mbr_meta

DatasetDict({
    train: Dataset({
        features: ['app_package', 'app_name', 'developer_name', 'app_category', 'description', 'content_rating', 'num_reviews', 'price', 'avg_rating'],
        num_rows: 10173
    })
})

In [6]:
#mbr_meta['train'].to_csv('./mbr_meta.csv')
apps_data = pd.read_csv('./mbr_meta.csv')

In [7]:
print("Columns:", apps_data.columns)
print("\nNumber of rows and columns:", apps_data.shape)
print("\nBasic Statistics for numerical data:")
print(apps_data.describe())
print("\nNumber of missing values in each column:")
print(apps_data.isnull().sum())

Columns: Index(['app_package', 'app_name', 'developer_name', 'app_category',
       'description', 'content_rating', 'num_reviews', 'price', 'avg_rating'],
      dtype='object')

Number of rows and columns: (10173, 9)

Basic Statistics for numerical data:
         avg_rating
count  10173.000000
mean       4.200020
std        0.561826
min        0.000000
25%        4.000000
50%        4.300000
75%        4.500000
max        5.000000

Number of missing values in each column:
app_package       0
app_name          0
developer_name    0
app_category      0
description       0
content_rating    0
num_reviews       0
price             0
avg_rating        0
dtype: int64


In [8]:
# How many unique app_packages (apps or items) are there?
print(f'There are {len(apps_data["app_package"].unique())} unique apps in app dataset.')

# How many categoris are there?
print(f'There are {len(apps_data["app_category"].unique())} unique categories in app dataset.')

There are 10173 unique apps in app dataset.
There are 48 unique categories in app dataset.


In [9]:
apps_data

,app_package,app_name,developer_name,app_category,description,content_rating,num_reviews,price,avg_rating
0,com.natewren.lines,Lines Pro - Icon Pack,Nate Wren Design,Personalization,Lines icons (pro version) are outlined shapes ...,Everyone,"2,997",$1.99 Buy,4.7
1,com.capitalone.credittracker,CreditWise from Capital One,"Capital One Services, LLC",Finance,Know your credit score and much more\n\nCapita...,Everyone,"94,914",Install,4.7
2,com.purascents.android,Pura - Smart Fragrance Dispenser,Pura,House & Home,Pura is a smart home fragrance device that you...,Everyone,778,Install,4.4
3,com.wallethub.mywallet,"WalletHub - Free Credit Score, Report & Monito...",WalletHub,Finance,WalletHub is the first app to offer 100% free ...,Everyone,"7,025",Install,4.7
4,air.com.samuramu.gg.se,#SelfLove (GG Confidence & Self esteem),Ggtude Ltd,Medical,Wish to build confidence and have higher self ...,Everyone,625,Install,4.1
...,...,...,...,...,...,...,...,...,...
10168,com.unysyegor.effects.neon.magicslate,Magic Slate - Neon Effects,Unysyegor,Entertainment,Magic Slate Neon Effects app contains on learn...,Everyone,0,Install,0.0
10169,com.romancestories.novel,Romance Stories-eBooks &Novels,Romance Stories Studio,Books & Reference,Romance Stories - Romantic Story & Fantasy Nov...,Teen,"2,591",Install,4.9
10170,com.cornerdesk.gfx.nstate,PUBG NEW STATE : GFX Tool Pro + 90FPS,CornerDesk Inc.,Libraries & Demo,GFX Tool is a free utility launcher for FPS ga...,Everyone,226,$2.49 Buy,1.0
10171,com.picture.magic.imager,Magic Photo Editor:Foto Repair,Zachary Holt,Photography,"Hi friends, the first photo editor to meet you...",Everyone,"2,724",Install,4.4


In [10]:
#Sampling and text combination for effective embeddings

apps_data_sample = apps_data[0:500]

apps_data_sample_refined =  apps_data[0:100]


apps_data_sample_refined['Combined'] = 'Name of the app is ' + apps_data_sample['app_name'] + '. App has been developed by ' + apps_data_sample['developer_name'] + '. It belongs to the category of ' + apps_data_sample['app_category'] + '. Following is a small description of the app: ' + apps_data_sample['description']



<ipython-input-10-d9e65f6629bb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apps_data_sample_refined['Combined'] = 'Name of the app is ' + apps_data_sample['app_name'] + '. App has been developed by ' + apps_data_sample['developer_name'] + '. It belongs to the category of ' + apps_data_sample['app_category'] + '. Following is a small description of the app: ' + apps_data_sample['description']


In [11]:
#%pip install OpenAI
#%pip install openai
import openai
#from google.colab import userdata

openai.api_key = "key"

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

#apps_data_sample["embedding_optimised"] = apps_data_sample['description'].apply(get_embedding)

apps_data_sample_refined["embedding_optimised"] = apps_data_sample_refined['Combined'].apply(get_embedding)

<ipython-input-11-08f8e1bc2cbc>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apps_data_sample_refined["embedding_optimised"] = apps_data_sample_refined['Combined'].apply(get_embedding)


In [12]:

apps_data_sample_refined

,app_package,app_name,developer_name,app_category,description,content_rating,num_reviews,price,avg_rating,Combined,embedding_optimised
0,com.natewren.lines,Lines Pro - Icon Pack,Nate Wren Design,Personalization,Lines icons (pro version) are outlined shapes ...,Everyone,"2,997",$1.99 Buy,4.7,Name of the app is Lines Pro - Icon Pack. App ...,"[-0.02398117445409298, -0.007237954530864954, ..."
1,com.capitalone.credittracker,CreditWise from Capital One,"Capital One Services, LLC",Finance,Know your credit score and much more\n\nCapita...,Everyone,"94,914",Install,4.7,Name of the app is CreditWise from Capital One...,"[0.013812982477247715, -0.008704314939677715, ..."
2,com.purascents.android,Pura - Smart Fragrance Dispenser,Pura,House & Home,Pura is a smart home fragrance device that you...,Everyone,778,Install,4.4,Name of the app is Pura - Smart Fragrance Disp...,"[-0.007991494610905647, 0.03432491421699524, -..."
3,com.wallethub.mywallet,"WalletHub - Free Credit Score, Report & Monito...",WalletHub,Finance,WalletHub is the first app to offer 100% free ...,Everyone,"7,025",Install,4.7,Name of the app is WalletHub - Free Credit Sco...,"[-0.018518265336751938, -0.01826835609972477, ..."
4,air.com.samuramu.gg.se,#SelfLove (GG Confidence & Self esteem),Ggtude Ltd,Medical,Wish to build confidence and have higher self ...,Everyone,625,Install,4.1,Name of the app is #SelfLove (GG Confidence & ...,"[-0.006071333307772875, -0.012674863450229168,..."
...,...,...,...,...,...,...,...,...,...,...,...
95,com.maxmpz.audioplayer.unlock,Poweramp Full Version Unlocker,Max MP,Music & Audio,"Music moves people; it unites us, it allows us...",Everyone,"280,641",$5.49 Buy,4.3,Name of the app is Poweramp Full Version Unloc...,"[-0.034622736275196075, 0.0017967626918107271,..."
96,com.rockstargames.bully,Bully: Anniversary Edition,Rockstar Games,Action,The Rockstar Games tradition of groundbreaking...,Teen,"44,920",$6.99 Buy,2.4,Name of the app is Bully: Anniversary Edition....,"[0.019980983808636665, 0.0029403232038021088, ..."
97,fm.app.moonbeam,Moonbeam | Podcast Discovery,Moonbeam.,Music & Audio,Moonbeam is the easiest way to discover your n...,Everyone,148,Install,4.1,Name of the app is Moonbeam | Podcast Discover...,"[0.018482143059372902, 0.004485695157200098, -..."
98,com.agilx.companycam,CompanyCam,CompanyCam,Productivity,"Manage your jobs, not your photos.\n\nCompanyC...",Everyone,519,Install,3.8,Name of the app is CompanyCam. App has been de...,"[0.019161829724907875, 0.019174177199602127, -..."


In [13]:
%pip install pymongo
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None

mongo_uri = "mongo server"
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client['apps']
collection = db['apps_data']

Connection to MongoDB successful


In [14]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 100, 'electionId': ObjectId('7fffffff0000000000000046'), 'opTime': {'ts': Timestamp(1711327376, 95), 't': 70}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1711327376, 104), 'signature': {'hash': b'\x85F \xcd\xd5\xed\xe0\xe7i\x1f|[\x8d\xcbX~\xb1\x1f\x99.', 'keyId': 7314681964852674562}}, 'operationTime': Timestamp(1711327376, 95)}, acknowledged=True)

In [15]:
documents = apps_data_sample_refined.to_dict('records')
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [28]:

def vector_search(user_query, collection):


    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding_optimised",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 5  # Return top 5 matches
            }
        },
        {
            "$project": {
                "app_package": 1,
                "app_name": 1,
                "app_category": 1,
                "developer_name": 1,
                "content_rating": 1,
                "price": 1,
                "avg_rating": 1,
                "score": {
                    "$meta": "vectorSearchScore"  # Include the search score
                }
            }
        }
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [17]:
def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
      search_result += f"App Package: {result.get('app_package', 'N/A')}, App Name : {result.get('app_name', 'N/A')} , App Category : {result.get('app_category', 'N/A')} , Developer: {result.get('developer_name', 'N/A')}, Content Rating: {result.get('content_rating', 'N/A')}, Price: {result.get('price', 'N/A')}, Average User Rating: {result.get('avg_rating', 'N/A')}, Score: {result.get('score', 'N/A')}\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a app recommendation system."},
          {"role": "user", "content": "Answer this user query: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result

In [18]:
# 6. Conduct query with retrival of sources
query = "Need app for creating videos"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response}")
print(f"Source Information: \n{source_information}")

Response: Based on your query, I recommend the "Video Maker" app by InShot Inc. The app falls under the category of Video Players & Editors, has a high average user rating of 4.8, and is suitable for users of all ages. It seems to be a popular choice for creating videos.
Source Information: 
App Package: videoeditor.videomaker.videoeditorforyoutube, App Name : Video Maker , App Category : Video Players & Editors , Developer: InShot Inc., Content Rating: Everyone, Price: Install, Average User Rating: 4.8, Score: 0.7902047634124756
App Package: com.agilx.companycam, App Name : CompanyCam , App Category : Productivity , Developer: CompanyCam, Content Rating: Everyone, Price: Install, Average User Rating: 3.8, Score: 0.7077385187149048
App Package: com.dreame.writer, App Name : Stary Writing-To Be A Stary Exclusive Writer , App Category : Books & Reference , Developer: Dreame Media, Content Rating: Teen, Price: Install, Average User Rating: 3.3, Score: 0.7055132985115051
App Package: com.f

In [19]:
!npm install localtunnel
!pip install streamlit

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.425s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [26]:
%%writefile app.py
import openai
from google.colab import userdata
import pymongo
import streamlit as st
import pandas as pd


openai.api_key = "key"
EMBEDDING_MODEL = "text-embedding-3-small"



def get_mongo_client(mongo_uri):
  """Establish connection to the MongoDB."""
  try:
    client = pymongo.MongoClient(mongo_uri)
    print("Connection to MongoDB successful")
    return client
  except pymongo.errors.ConnectionFailure as e:
    print(f"Connection failed: {e}")
    return None


apps_data = pd.read_csv('./mbr_meta.csv')
apps_data_sample_refined =  apps_data[0:100]
apps_data_sample_refined['Combined'] = 'Name of the app is ' + apps_data_sample_refined['app_name'] + '. App has been developed by ' + apps_data_sample_refined['developer_name'] + '. It belongs to the category of ' + apps_data_sample_refined['app_category'] + '. Following is a small description of the app: ' + apps_data_sample_refined['description']


#Connection

mongo_uri = "momgoserver"
if not mongo_uri:
  print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client['apps']
collection = db['apps_data']




#Get Embedding
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

apps_data_sample_refined["embedding_optimised"] = apps_data_sample_refined['Combined'].apply(get_embedding)



collection.delete_many({})



documents = apps_data_sample_refined.to_dict('records')
collection.insert_many(documents)
print("Data ingestion into MongoDB completed")



def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding_optimised",
                "numCandidates": 25,  # Number of candidate matches to consider
                "limit": 5  # Return top 5 matches
            }
        },
        {
            "$project": {
                "app_package": 1,
                "app_name": 1,
                "app_category": 1,
                "developer_name": 1,
                "content_rating": 1,
                "price": 1,
                "avg_rating": 1,
                "score": {
                    "$meta": "vectorSearchScore"  # Include the search score
                }
            }
        }
    ]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)


def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  search_result = ''
  for result in get_knowledge:
      search_result += f"App Package: {result.get('app_package', 'N/A')}, App Name : {result.get('app_name', 'N/A')} , App Category : {result.get('app_category', 'N/A')} , Developer: {result.get('developer_name', 'N/A')}, Content Rating: {result.get('content_rating', 'N/A')}, Price: {result.get('price', 'N/A')}, Average User Rating: {result.get('avg_rating', 'N/A')}, Score: {result.get('score', 'N/A')}\n"

  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a app recommendation system."},
          {"role": "user", "content": "Answer this user query: " + query + " with the following context: " + search_result}
      ]
  )

  return (completion.choices[0].message.content), search_result

# 6. Conduct query with retrival of sources

st.set_page_config(page_title="App Recommendation System")
st.header("What app are you looking for?")
query = st.text_input("Query")
if query != "":
  response, source_information = handle_user_query(query, collection)
  print(source_information)
  print(response)
  st.write(f"Response:\n{response}\n\nSource Information:\n{source_information}")

Overwriting app.py


In [27]:
!npx localtunnel --port 8501 & streamlit run app.py & curl ipv4.icanhazip.com

  Stopping...


In [ ]:
!curl https://loca.lt/mytunnelpassword

In [ ]:
!wget -q -O - https://loca.lt/mytunnelpassword